In [1]:
import pandas as pd
import sys
from top2vec import Top2Vec
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import punkt
from nltk.stem import WordNetLemmatizer
from collections import Counter
import umap.umap_ as umap
import umap.plot
nltk.download("stopwords")
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
import spacy
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim_models
from bertopic import BERTopic
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\graph_objs\__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages

In [2]:

def explore_common_words(df):
    stop = stopwords.words("english")
    df = df.astype(str)
    df["allegation_desc"] = df["allegation_desc"].apply(
        lambda x: " ".join([word for word in x.split() if word not in (stop)])
    )

    df.loc[:, "topics"] = " ".join(x for x in df["allegation_desc"].astype(str))
    df = df["topics"][0]
    df = df.split()

    counts = Counter(df)
    most_occur = counts.most_common(50)
    print(most_occur)
    return df

In [3]:
# def preprocess_nopd(df):
#     df.loc[:, "allegation_desc"] = df.allegation_desc.astype(str).fillna("")\
#         .str.replace(r"( ?officers? | ?complain?a?n?ts?\.? ?| ?stated ?| ?accused ?| ?nan ?| ?police ?)", "", regex=True)
#     return df

In [4]:
def read_data():
    df = pd.read_csv("../data/training_data/complaint_classification_labeled_data_1_31_2023.csv")
    return df

In [5]:
df = read_data()

In [6]:
def clean_labels(df):
    df.loc[:, "label"] = df.label.str.lower().str.strip().str.replace(r"#", "/", regex=False)
    return df 

In [7]:
df = df.pipe(clean_labels)

In [8]:
df.label.unique()

array(['inadequate description', 'force',
       'harm to the general public safety',
       'arrests/stops/searches/evidence',
       'internal misconduct/administrative infractions', 'traffic',
       'arrests/stops/searches', 'evidence',
       'evidence/harm to the general public safety', 'camera', 'juvenile',
       'sexual', 'inmate/corrections', 'evidence/inmate/corrections',
       'harm to the general public safety/internal misconduct/administrative infractions',
       'force/inmate/corrections', 'evidence/force',
       'camera/internal misconduct/administrative infractions',
       'camera/force',
       'force/internal misconduct/administrative infractions',
       'arrests/stops/searches/camera/discrimination',
       'camera/inmate/corrections',
       'arrests/stops/searches/internal misconduct/administrative infractions',
       'arrests/stops/searches/traffic', nan, 'force/traffic', 'court',
       'domestic', 'domestic/force', 'arrests/stops/searches/sexual',
       

In [9]:
def extract_labels(df):
    dfa = df
    dfa_targets = df.label.str.lower().str.strip().str.extract(r"(traffic|evidence|arrests\/stops\/searches)")
    dfa.loc[:, "target"] = dfa_targets[0].str.replace(r"(.+)", "0", regex=True)
    dfa = dfa[~((dfa.target.fillna("") == ""))]

    targets = df.label.str.lower().str.strip().str.extract(r"(internal misconduct\/administrative infractions)")

    df.loc[:, "target"] = targets[0].str.replace(r"(internal misconduct\/administrative infractions)", "1", regex=True)
    df = df[~((df.target.fillna("") == ""))]

    df = pd.concat([df, dfa], axis=0)

    return df.rename(columns={"text": "allegation_desc"})

df = df.pipe(extract_labels)
df

,id,allegation_desc,meta_agency,meta_tracking_id,label,target
13,13645,the former chief of police received a complain...,benton-pd,NaN,internal misconduct/administrative infractions,1
29,13691,alleges officer burr made inappropriate facebo...,bossier-city-pd,20-IA-21,internal misconduct/administrative infractions,1
39,13725,refused an assignment.,new-orleans-so,C-009-20,internal misconduct/administrative infractions,1
40,13726,left work without informing her rank.,new-orleans-so,G-043-20,internal misconduct/administrative infractions,1
46,13744,failed to inform his supervisor that he would ...,new-orleans-so,H-024-20,harm to the general public safety/internal mis...,1
...,...,...,...,...,...,...
982,17938,the complainant alleged the accused officers d...,new-orleans-pd,2020-0307-p,arrests/stops/searches/juvenile,0
983,17946,officer continued in the same direction of a f...,new-orleans-pd,2020-0332-r,traffic,0
985,17955,complainant alleges nopd did not send him a ce...,new-orleans-pd,2020-0368-p,evidence,0
987,17964,accused an officer of being aggressive and unp...,new-orleans-pd,2020-0397-r,evidence/internal misconduct/administrative in...,0


In [10]:
def shuffle_data(df):
    df = df.astype(str)
    df = df.sample(frac=1)
    return df

In [11]:
df = df.pipe(shuffle_data)

In [12]:
df

,id,allegation_desc,meta_agency,meta_tracking_id,label,target
27,13689,"alleges officer moses was umprofessional, hand...",bossier-city-pd,20-14-35,arrests/stops/searches,0
936,17731,accused caused damage to complainant's vehicle...,new-orleans-pd,2019-0584-p,traffic,0
417,15382,officer failed to report that he was involved ...,new-orleans-pd,2015-0396-r,traffic,0
779,17028,the complainant stated he was stopped for a tr...,new-orleans-pd,2017-0723-p,traffic,0
798,17110,complainant accused officers of damaging her p...,new-orleans-pd,2018-0168-p,arrests/stops/searches,0
...,...,...,...,...,...,...
517,15832,officer failed to receive permission from an o...,new-orleans-pd,2016-0220-d,internal misconduct/administrative infractions,1
192,14373,refused to leave pod after receiving instructi...,new-orleans-so,L-033-19,internal misconduct/administrative infractions,1
213,14464,reported 6 minutes late for work.,new-orleans-so,B-018-19,internal misconduct/administrative infractions,1
722,16768,accused was observed sleeping in his marked po...,new-orleans-pd,2017-0415-d,harm to the general public safety/internal mis...,1


In [13]:
def split_data(df):
    training_data = df[:300]
    test_data = df[300:]
    return training_data, test_data

In [14]:
training_data, test_data = split_data(df)

In [15]:
################################################### top2vec ##################################################################################

In [16]:
def create_model(training_data):
    # unique = [x for x in df["allegation_topic_uid"]]
    training_data = [x for x in training_data["allegation_desc"]]

    model = Top2Vec(
        training_data,
        ngram_vocab=True,
        speed="deep-learn",
        use_embedding_model_tokenizer=True,
        min_count=5,
        # document_ids=unique
    )
    return model

In [17]:
def model():
    top2vec = create_model(training_data)
    return top2vec

In [18]:
model = model()

2023-01-31 12:47:16,254 - top2vec - INFO - Pre-processing documents for training
2023-01-31 12:47:16,291 - top2vec - INFO - Creating joint document/word embedding
2023-01-31 12:47:27,177 - top2vec - INFO - Creating lower dimension embedding of documents
2023-01-31 12:47:47,207 - top2vec - INFO - Finding dense areas of documents
2023-01-31 12:47:47,266 - top2vec - INFO - Finding topics


In [19]:
if len(model.get_topic_sizes()) > 1:
    topic_words, word_scores, topic_nums = model.get_topics()
    for words, scores, num in zip(topic_words, word_scores, topic_nums):
        print(num)
        print(f"Words: {words}")

0
Words: ['call' 'refused' 'perform' 'log' 'detective' 'failed' 'neglect'
 'document' 'may' 'report' 'service' 'force' 'pursuit' 'off' 'violation'
 'no' 'accident' 'involved' 'have been' 'taking' 'sick' 'manner' 'rude'
 'against' 'violated' 'employee' 'detail' 'money' 'notify' 'unauthorized'
 'write' 'instructions' 'assigned' 'complete' 'their' 'accused' 'duty'
 'conduct' 'allegation' 'failing' 'hours' 'action' 'summons' 'officers'
 'evidence' 'incident' 'investigation' 'during' 'unprofessional' 'leaving']
1
Words: ['party' 'station' 'into' 'hit' 'scene' 'arrived' 'didn' 'check' 'another'
 'threatened' 'who' 'other' 'time' 'lights' 'or' 'he' 'unit' 'told'
 'district' 'because' 'about' 'over' 'been' 'have' 'get' 'issue' 'subject'
 'called' 'were' 'his' 'that' 'stated' 'sergeant' 'out' 'minutes'
 'observed' 'late' 'should' 'so' 'stolen' 'car' 'complainant' 'it' 'they'
 'unknown' 'girlfriend' 'any' 'rear' 'him' 'if']
2
Words: ['pib' 'vehicle pursuit' 'traffic stop' 'break' 'marked' 'drive

In [20]:
topic_sizes, top_nums = model.get_topic_sizes()
print(topic_sizes)
print(top_nums)

[125 103  72]
[0 1 2]


In [21]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=1, num_docs=10)

for doc, score, doc_id in list(zip(documents, document_scores, document_ids)):
    print(f"Document: {doc_id}, Score: {score}")
    print("--------------------")
    print(doc)
    print("--------------------")

Document: 256, Score: 0.9751224517822266
--------------------
officer didn’t issue other party a ticket
--------------------
Document: 115, Score: 0.9451191425323486
--------------------
attempted to end her shift early to attend granddaughter's party.
--------------------
Document: 73, Score: 0.9312306642532349
--------------------
the accused officers were observed working unauthorized details.
--------------------
Document: 97, Score: 0.9219014048576355
--------------------
accused of not providing any data for weekly statistics.
--------------------
Document: 111, Score: 0.9217131733894348
--------------------
complainant stated an officer falsely arrested him.
--------------------
Document: 79, Score: 0.9125678539276123
--------------------
the complainant alleged the accused officers demeaned her by arresting her in front of her son after they falsely stated they were doing a welfare check on the child.
--------------------
Document: 9, Score: 0.900023341178894
------------------

In [22]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=0, num_docs=20)

ents = {(doc, score) for doc, score in list(zip(documents, document_scores))}
df = pd.DataFrame(ents, columns=["allegation_desc", "score"])
print(df)
# df.loc[:, "topic"] = "9"

                                      allegation_desc     score
0   alleges officer burr made inappropriate facebo...  0.913855
1   complainant accused officer of lying about hav...  0.955019
2   accused of taking deputy's commission during d...  0.949234
3   complainant alleged an employee was directly i...  0.876545
4   accused of speaking to a cbm employee in a dis...  0.963837
5   accused failed document a c.l't. form on a cal...  0.951045
6   accused may have known one of the drivers whil...  0.883197
7   complainant stated that the accused lost evide...  0.875465
8   the officer committing several driving violati...  0.871866
9                      failed to properly store pans.  0.899162
10      failed to clock out when instructed to do so.  0.870576
11  complainant was involved in a traffic accident...  0.896750
12  investigation cancelled  ‘the officer failed t...  0.903811
13  officers were accused of not documenting evide...  0.875397
14  accused failed to complete task as p

In [23]:
model.save("models/noso")

In [24]:
# model.generate_topic_wordcloud(0)

In [25]:
model.topic_words

array([['call', 'refused', 'perform', 'log', 'detective', 'failed',
        'neglect', 'document', 'may', 'report', 'service', 'force',
        'pursuit', 'off', 'violation', 'no', 'accident', 'involved',
        'have been', 'taking', 'sick', 'manner', 'rude', 'against',
        'violated', 'employee', 'detail', 'money', 'notify',
        'unauthorized', 'write', 'instructions', 'assigned', 'complete',
        'their', 'accused', 'duty', 'conduct', 'allegation', 'failing',
        'hours', 'action', 'summons', 'officers', 'evidence', 'incident',
        'investigation', 'during', 'unprofessional', 'leaving'],
       ['party', 'station', 'into', 'hit', 'scene', 'arrived', 'didn',
        'check', 'another', 'threatened', 'who', 'other', 'time',
        'lights', 'or', 'he', 'unit', 'told', 'district', 'because',
        'about', 'over', 'been', 'have', 'get', 'issue', 'subject',
        'called', 'were', 'his', 'that', 'stated', 'sergeant', 'out',
        'minutes', 'observed', 'late',

In [26]:
# model.topic_words[0]

In [27]:
topic_words, word_scores, topic_nums = model.get_topics(0)
for words, scores, nums in zip(topic_words, word_scores, topic_nums):
  print("Topic Number: ",nums)
  print(f"Words: {words}")
  print("\n")

In [28]:
# topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["inmate"], num_topics=0)
# for word, w_score, topic, t_score in list(zip(topic_words, word_scores, topic_scores, topic_nums)):
#     print(f"Word: \n{word}")
#     print("--------------------")
#     print(f"Word Score \n{w_score}")
#     print("--------------------")
#     print(f"Topic Score: \n{topic}")
#     print("--------------------")
#     print (f"Topic # \n{t_score}")
#     print("--------END---------")

In [29]:
# model = Top2Vec.load("models/noso")

# umap_args = {
#     "n_neighbors": 15,
#     "n_components": 2, # 5 -> 2 for plotting 
#     "metric": "cosine",
# }
# umap_model = umap.UMAP(**umap_args).fit(model.topic_vectors)
# umap.plot.points(umap_model, labels=model.doc_top_reduced)

In [30]:
############################################################ bert #################################################################

In [31]:
docs = training_data.allegation_desc

In [32]:
docs = json.loads(docs.to_json(orient='records'))

In [33]:
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")

In [34]:
topic, probs = topic_model.fit_transform(docs)

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\astroid\node_classes.py:94: DeprecationWarning: The 'astroid.node_classes' module is deprecated and will be replaced by 'astroid.nodes' in astroid 3.0.0
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,0,289,0_the_to_complainant_officer
1,1,11,1_late_reported_for_duty


In [ ]:
# topic_model.get_topic(3)

In [ ]:
# topic_model.get_representative_docs(3)

In [ ]:
# topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\io\_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [ ]:
df_bert = pd.DataFrame({"topic": topic, "documents": docs})

In [ ]:
df_bert

,topic,documents
0,0,accused stated he would make a false accusatio...
1,0,the officer failed to attend re-qualification ...
2,0,complainant states she was involved in an acci...
3,0,failed to serve papers.
4,0,the accused officer was going to therapy for a...
...,...,...
295,0,found to be in possession of cell phone on pod.
296,0,accused an officer of being aggressive and unp...
297,0,the complainant alleged the accused officers d...
298,0,investigation cancelled ‘the officer failed t...


In [ ]:
############################################################ gensim ##############################################################################

In [ ]:
train_docs = training_data.allegation_desc

In [ ]:
def lemmatization(descs, allowed_pos_tags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    final_text = []
    for desc in descs:
        doc = nlp(desc)
        new_text = " ".join([token.lemma_ for token in doc if token.pos_ in allowed_pos_tags])
        final_text.append(new_text)
    return (final_text)

In [ ]:
lemmatized_texts = lemmatization(docs)

In [ ]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [ ]:
data_words = gen_words(lemmatized_texts)

In [ ]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return list(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return list(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

# print(data_bigrams_trigrams[5])

In [ ]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

train_corpus = [id2word.doc2bow(text) for text in texts]
# print(train_corpus[0][0:20])

tdidf = TfidfModel(train_corpus, id2word=id2word)

low_value = 0.03
words = []
words_missing_in_tdif = []

for i in range(0, len(train_corpus)):
    bow = train_corpus[i]
    low_value_words = []
    tdif_ids = [id for id, value in tdidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tdidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tdif
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tdif = [id for id in bow_ids if id not in tdif_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tdif]
    train_corpus[i] = new_bow

In [ ]:
# id2word = corpora.Dictionary(data_words)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=train_corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                        #    per_word_topics=True,
                                           alpha="auto")
lda_model.save('models/lda_train.model')

In [ ]:
# lda_model.print_topics(5, num_words=20)[:10]


In [ ]:
def lda_vecs():
    lda_train_vecs = []
    for i in range(len(train_docs)):
        top_topics = lda_model.get_document_topics(train_corpus[i], minimum_probability=0.0)
        topic_vec = [top_topics[i][1] for i in range(20)]
        topic_vec.extend([len(train_docs.iloc[i])]) # length review
        lda_train_vecs.append(topic_vec)
    return lda_train_vecs

In [ ]:
lda_vecs = lda_vecs()

In [ ]:
lda_vecs

[[0.009795464,
  0.0024657955,
  0.0023802146,
  0.0047121407,
  0.0060236594,
  0.004667829,
  0.0038166624,
  0.30929884,
  0.119085826,
  0.006232716,
  0.4678071,
  0.013344787,
  0.004131879,
  0.0037494064,
  0.0114792725,
  0.011774034,
  0.005973813,
  0.0075005735,
  0.0026973386,
  0.0030626336,
  84],
 [0.011190709,
  0.002817022,
  0.002719251,
  0.005383335,
  0.006881666,
  0.0053327116,
  0.0043603056,
  0.007782408,
  0.005488876,
  0.0071205003,
  0.013595548,
  0.015241915,
  0.004720422,
  0.0042834696,
  0.013113409,
  0.2048272,
  0.52756065,
  0.008568952,
  0.003081546,
  0.1459301,
  61],
 [0.007129004,
  0.0017945528,
  0.001732269,
  0.0034293942,
  0.0043838895,
  0.003397145,
  0.0027776845,
  0.004957698,
  0.0034966278,
  0.19636922,
  0.008659721,
  0.7252164,
  0.0030070927,
  0.002728737,
  0.008353717,
  0.008568518,
  0.004347612,
  0.0054587545,
  0.001963065,
  0.0022289187,
  120],
 [0.01953892,
  0.004918569,
  0.004747859,
  0.009399396,
  0.0120

In [ ]:
# test_doc = train_corpus[-1]

# vector = lda_model[test_doc]

# def Sort(sub_li):
#     sub_li.sort(key = lambda x: x[1])
#     sub_li.reverse()
#     return (sub_li)

# new_vector = Sort(vector)
# print(new_vector)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, train_corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.346875 -0.076494       1        1  15.876440
14    -0.318590  0.069362       2        1  13.850682
15    -0.237678 -0.197148       3        1  13.784251
10    -0.144624 -0.272983       4        1  10.351891
0     -0.183424  0.256917       5        1   9.589596
4     -0.175616  0.165600       6        1   5.216539
17    -0.139768  0.013138       7        1   5.201478
9     -0.048698 -0.191418       8        1   4.528231
7      0.134981 -0.269705       9        1   4.491743
16     0.233921 -0.113990      10        1   3.578320
3      0.120579 -0.147593      11        1   3.152317
5      0.268768  0.011221      12        1   2.495604
12     0.039592 -0.067751      13        1   2.055635
8      0.183138  0.205883      14        1   1.905389
6      0.103492  0.196197      15        1   1.179884
13     0.031576  0.117231      16        1   1.134019
19     0.158623  0.075074      17        1   0.662161
18     0.113958  0.090887      18        1   0.516841
1      0.104537  0.068362      19        1   0.241543
2      0.102107  0.067209      20        1   0.187435, topic_info=            Term        Freq       Total Category  logprob  loglift
8           fail  109.000000  109.000000  Default  30.0000  30.0000
6          state  114.000000  114.000000  Default  29.0000  29.0000
68       vehicle   57.000000   57.000000  Default  28.0000  28.0000
13   complainant  210.000000  210.000000  Default  27.0000  27.0000
298         call   33.000000   33.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
25         clock    0.007294    2.168173  Topic20  -6.8035   0.5848
26      continue    0.007294    2.490841  Topic20  -6.8035   0.4461
27     discharge    0.007294    1.937374  Topic20  -6.8035   0.6974
28            go    0.007294   10.721087  Topic20  -6.8035  -1.0135
29       however    0.007294    3.234208  Topic20  -6.8035   0.1849

[856 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
12        1  0.495588    accident
12        3  0.419344    accident
12        5  0.076244    accident
288       1  0.635592      accord
196       4  0.673146  accordance
...     ...       ...         ...
36       11  0.032949        work
292       1  0.942752       write
320       3  0.877736  wrongfully
334      17  0.508187        year
413      13  0.796151       yield

[614 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 15, 16, 11, 1, 5, 18, 10, 8, 17, 4, 6, 13, 9, 7, 14, 20, 19, 2, 3])